In [1]:
import pandas as pd
import h3


df = pd.read_csv("../data/raw/final_sales_data_CLEANED.csv", encoding="utf-8-sig")



C:\Users\ACER\AppData\Local\Temp\ipykernel_23944\4058894971.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/final_sales_data_CLEANED.csv", encoding="utf-8-sig")


In [2]:
pd.set_option('display.max_columns', None)
# print(df['building_type'].value_counts())
# df.info()
# df.head()

In [3]:

#gereksizleri tullayiram
df = df.drop(columns=["createdAt", "updatedAt", "building_type", "all_pictures", "completion_status","purpose", "hasProject"])
df.columns = [col.lower() for col in df.columns]

# NOT: Sütun sıralaması tüm veri zenginleştirme (enrichment) 
# işlemlerinden sonra en sonda toplu olarak yapılacaktır.

In [4]:
#Numerical olan, lakin categorical olmali olan sutunlari categorical edirem. Reactivate date xaric, o date typedi. 
df = df.astype({
    'id': 'str',
    'rooms': 'Int64',
})

for col in ['building_no', 'postal_code', 'additional_no', 'deed_number']:
    df[col] = pd.to_numeric(df[col], errors="coerce").round().astype('Int64').astype('str')
    df[col] = df[col].replace(['nan','<NA>'], '')
    # print(df[col].value_counts())


df['reactivatedat'] = pd.to_datetime(df['reactivatedat'])
# df.info()
# df.head()

In [5]:
df.info()
df.describe().round(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18315 entries, 0 to 18314
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      18315 non-null  object        
 1   name                    18314 non-null  object        
 2   price                   18315 non-null  float64       
 3   rooms                   18315 non-null  Int64         
 4   bath                    18315 non-null  int64         
 5   area_m2                 18315 non-null  float64       
 6   location                18315 non-null  object        
 7   lat                     18315 non-null  float64       
 8   long                    18315 non-null  float64       
 9   building_no             18315 non-null  object        
 10  amenities               18247 non-null  object        
 11  furnished               18315 non-null  object        
 12  reactivatedat           18315 non-null  dateti

,price,rooms,bath,area_m2,lat,long,reactivatedat,beds
count,18315.00,18315.0,18315.00,18315.00,18315.00,18315.00,18315,18315.00
mean,813356.27,3.71,2.18,340.54,23.18,42.98,2025-10-13 11:05:47.321867520,2.45
min,350.00,1.0,0.00,24.73,21.41,38.96,2025-01-12 00:00:00,0.00
25%,550000.00,3.0,0.00,122.99,21.59,39.16,2025-09-01 00:00:00,0.00
50%,750000.00,4.0,3.00,146.17,24.50,46.50,2025-10-24 00:00:00,3.00
75%,950000.00,4.0,3.00,208.57,24.66,46.63,2025-12-12 00:00:00,4.00
max,13000000.00,345.0,11.00,103104.38,25.00,46.98,2026-01-07 00:00:00,11.00
std,582611.34,2.86,1.56,1770.79,1.55,3.74,NaN,1.82


In [6]:
#numerical, categorical, identical ve zaman olaraq 4 yere bolurem
num_cols = ['price', 'rooms','beds','bath', 'area_m2', 'lat', 'long']

cat_cols = ['location', 'amenities', 'furnished', 'city', 'region', 
                    'district', 'type', 'listing_age', 'listing_face', 
                    'is_listing_constrained', 'is_listing_pawned', 
                    'agency_name', 'residence_type', 'isverified']

date_cols = ['reactivatedat']

id_cols = ['id', 'name', 'link', 'building_no', 'postal_code','deed_number', 'additional_no']

In [7]:
#her Numerical sütunun min ve max deyerini, deyerlerin sayını ve missing deyerlerin sayını gösterir.  
num_summary = df[num_cols].describe().loc[['min', 'max', 'count']]

num_summary.loc['missing'] = df[num_cols].isnull().sum()

num_summary


,price,rooms,beds,bath,area_m2,lat,long
min,350.0,1.0,0.0,0.0,24.73,21.411700,38.95625
max,13000000.0,345.0,11.0,11.0,103104.38,25.000033,46.98135
count,18315.0,18315.0,18315.0,18315.0,18315.00,18315.000000,18315.00000
missing,0.0,0.0,0.0,0.0,0.00,0.000000,0.00000


In [8]:
#categorical sütunların missing deyerlerinin sayını, benzersiz deyerlerin sayını ve hemin sütunda ən çox olan deyeri gösterir
cat_summary = pd.DataFrame({
    'missing': df[cat_cols].isnull().sum(),
    'unique_count': df[cat_cols].nunique(),
    'frequency': df[cat_cols].mode().iloc[0]
})

cat_summary

,missing,unique_count,frequency
location,0,157,Al Mahdiyah
amenities,68,3239,"Electricity, Water Supply, Sewerage"
furnished,0,2,unfurnished
city,0,2,Riyadh
region,17,14,North Jeddah
district,273,148,Al Mahdiyah
type,1285,2,agency
listing_age,0,13,New
listing_face,9121,10,Eastern
is_listing_constrained,0,2,No


In [9]:

def get_h3_index(row):
    return h3.latlng_to_cell(row['lat'], row['long'], res=9)

df['h3_index'] = df.apply(get_h3_index, axis=1)

df['h3_11'] = df.apply(
    lambda row: h3.latlng_to_cell(
        row['lat'],
        row['long'],
        11
    ),
    axis=1
)

# df

In [10]:
# Bilimsel Global Filtreler (Fiyat, Oda, Alan)
# 1. Oda: 1-15 (Apartment icin normal aralik)
# 2. Fiyat: 2000 - 3M SAR (Cok ucuz veya pahali aykiri degerleri atar)
# 3. Alan: 30 - 2000 m2 (Asiri kucuk veya arsa buyuklugundeki yerleri atar)
df = df[
    (df['rooms'].between(1, 15)) & 
    (df['price'].between(2000, 3000000)) & 
    (df['area_m2'].between(30, 2000))
]

# 1. Her benzersiz tapu numarasına bir Property ID ver
# df['property_id'] = df.groupby('deed_number').ngroup()
# Aynı bina + aynı oda sayısı = aynı daire tipi
df['property_id'] = df.groupby(['deed_number', 'rooms', 'h3_11']).ngroup()


# 2. Her mülk için toplam ilan sayısını hesapla (Kaç tane mükerrer olduğunu görmek için)
df['total_listings_count'] = df.groupby('property_id')['id'].transform('count')

# df

In [11]:
# Her mülk grubu için 'doğru' kabul edilecek değerleri (medyan) hesapla
# Bu işlem uç değerleri (345 oda vb.) otomatik olarak etkisiz hale getirir
group_medians = df.groupby('property_id')[num_cols].transform('median')



# Mevcut değerler medyan ile uyuşuyor mu? (Tolerans: Alan için %10, diğerleri birebir aynı olmalı)
df['is_faulty'] = (
    # (df['rooms'] != group_medians['rooms']) | 
    (df['area_m2'] > group_medians['area_m2'] * 1.1) | 
    (df['area_m2'] < group_medians['area_m2'] * 0.9)
)



print(f"Hatalı/Yalan bilgi içeren ilan sayısı: {df['is_faulty'].sum()}")
df.describe()
# df


Hatalı/Yalan bilgi içeren ilan sayısı: 23


,price,rooms,bath,area_m2,lat,long,reactivatedat,beds,property_id,total_listings_count
count,1.785500e+04,17855.0,17855.000000,17855.000000,17855.000000,17855.000000,17855,17855.000000,17855.000000,17855.000000
mean,7.851611e+05,3.681994,2.202688,260.917268,23.194899,43.014580,2025-10-13 05:36:23.388406528,2.479474,3763.775077,10.178381
min,1.800000e+04,1.0,0.000000,42.860000,21.411700,38.956250,2025-01-12 00:00:00,0.000000,0.000000,1.000000
25%,5.600000e+05,3.0,0.000000,122.190000,21.588321,39.164411,2025-09-01 00:00:00,0.000000,1857.000000,1.000000
50%,7.500000e+05,4.0,3.000000,145.550000,24.507089,46.499509,2025-10-24 00:00:00,3.000000,3912.000000,4.000000
75%,9.500000e+05,4.0,3.000000,203.160000,24.663642,46.630659,2025-12-12 00:00:00,4.000000,5682.500000,13.000000
max,3.000000e+06,13.0,11.000000,2000.000000,25.000033,46.981350,2026-01-07 00:00:00,11.000000,7421.000000,94.000000
std,3.259178e+05,1.047285,1.561112,281.206814,1.551378,3.735457,NaN,1.821056,2177.019801,15.239988


In [12]:
# Her mülk içindeki ilanları tarihe göre sırala ve en güncelini 'Ana İlan' seç
df['is_main_listing'] = False
main_indices = df.sort_values(['property_id', 'reactivatedat'], ascending=[True, False]).groupby('property_id').head(1).index
df.loc[main_indices, 'is_main_listing'] = True
# df

In [13]:
# Sadece ana ilanları al ve hatalı işaretlenenleri (outliers) dışarıda bırak
df_clean = df[(df['is_main_listing'] == True) & (df['is_faulty'] == False)].copy()

# Temizlik sonrası veri kontrolü
print(f"Başlangıç ilan sayısı: {len(df)}")
print(f"Temizlik sonrası gerçek mülk sayısı: {len(df_clean)}")
print(f"Yeni Maksimum Oda Sayısı: {df_clean['rooms'].max()}") # Artık 345 görmemelisin
df_clean.describe()
# df_clean

Başlangıç ilan sayısı: 17855
Temizlik sonrası gerçek mülk sayısı: 7417
Yeni Maksimum Oda Sayısı: 13


,price,rooms,bath,area_m2,lat,long,reactivatedat,beds,property_id,total_listings_count
count,7.417000e+03,7417.0,7417.000000,7417.000000,7417.000000,7417.000000,7417,7417.000000,7417.000000,7417.000000
mean,8.244159e+05,3.826075,2.320615,220.356021,23.449259,43.619406,2025-10-13 08:46:31.856545792,2.619523,3708.867736,2.405959
min,2.500000e+04,1.0,0.000000,42.860000,21.411700,38.956250,2025-01-20 00:00:00,0.000000,0.000000,1.000000
25%,5.700000e+05,3.0,0.000000,120.980000,21.606991,39.198249,2025-08-27 00:00:00,0.000000,1854.000000,1.000000
50%,7.400000e+05,4.0,3.000000,146.200000,24.554096,46.554893,2025-10-27 00:00:00,3.000000,3708.000000,1.000000
75%,1.000000e+06,5.0,3.000000,190.100000,24.807567,46.751501,2025-12-12 00:00:00,4.000000,5563.000000,2.000000
max,3.000000e+06,13.0,11.000000,2000.000000,25.000033,46.981350,2026-01-07 00:00:00,11.000000,7421.000000,94.000000
std,3.721518e+05,1.149764,1.595976,230.313300,1.541328,3.702728,NaN,1.907527,2142.469781,4.325931


In [14]:
# # Bilimsel ve piyasa gerçeklerine dayalı nihai filtreleme
# df_clean = df_clean[
#     (df_clean['price'] >= 100000) &      # 100k altı satış verisi için gerçekçi değildir
#     (df_clean['rooms'] <= 15) &         # 15 odadan fazlası 'Apartment' kategorisi için hatadır
#     (df_clean['area_m2'] >= 20) &       # 20 m2 altı ev değildir
#     (df_clean['area_m2'] <= 1500)       # 1500 m2 üstü genelde hatalı arsa verisidir
# ].copy()

# print(f"Global temizlik sonrası kalan mülk sayısı: {len(df_clean)}")
# print(f"Yeni Max Oda: {df_clean['rooms'].max()} | Yeni Min Fiyat: {df_clean['price'].min()}")
# df_clean.describe()


In [15]:
# 1. Önce veriyi tarihe göre sıralayalım ki 'drop_duplicates' en güncel ilanı tutsun
df_final = df_clean.sort_values(by='reactivatedat', ascending=False)

# 2. Hem Hard Match hem Soft Match'i kapsayan nihai tekilleştirme
# Aynı konum, aynı m2 ve aynı oda sayısına sahip her şeyi teke indiriyoruz
df_final = df_final.drop_duplicates(
    subset=['h3_index', 'area_m2', 'rooms', 'bath'], 
    keep='first'
)

print(f"Temizlik Öncesi (Ham): 18.315")
print(f"Hard Match + Outlier Sonrası: {len(df_clean)}") 
print(f"Soft Match (NİHAİ) Sonrası: {len(df_final)}")
# df_final


Temizlik Öncesi (Ham): 18.315
Hard Match + Outlier Sonrası: 7417
Soft Match (NİHAİ) Sonrası: 6837


In [16]:
# --- AMENITIES PARSING (YENI: AI & FIYAT ODAKLI) ---
import pandas as pd
from collections import Counter

# Sadece frekansa değil, fiyata etki edebilecek lüks özelliklere de odaklanalım
luxury_important = [
    'Pool', 'Garden', 'Gym', 'Smart Home', 'Maid', 'Driver', 
    'Security', 'Elevator', 'Balcony', 'Terrace', 'View'
]

# Mevcut özellikleri ayıkla
df_final['amenities'] = df_final['amenities'].fillna('')
all_raw_items = []
for row in df_final['amenities']:
    items = [i.strip() for i in row.split(',') if i.strip()]
    all_raw_items.extend(items)

counts = Counter(all_raw_items)
# En sık geçen 15 özellik + listedeki önemli lüks özellikler
top_freq = [item for item, count in counts.most_common(15)]
all_target_amenities = list(set(top_freq + luxury_important))

# Hassas eşleştirme (str.contains yerine liste kontrolü)
def check_amenity(row_text, target):
    row_list = [i.strip().lower() for i in row_text.split(',') if i.strip()]
    target_lower = target.lower()
    # Hem tam kelime eşleşmesi hem de içerme kontrolü (Pool vs Swimming Pool için)
    return any(target_lower in item for item in row_list)

for am in all_target_amenities:
    col_name = 'has_' + am.lower().replace(' ', '_').replace('/', '_')
    df_final[col_name] = df_final['amenities'].apply(lambda x: check_amenity(x, am))

# Temasel gruplar oluşturalım (Gereksiz sütun kalabalığını azaltmak için)
df_final['has_luxury'] = df_final[['has_pool', 'has_garden', 'has_gym', 'has_smart_home']].any(axis=1)

print(f"Toplam {len(all_target_amenities)} özellik parse edildi.")
print("Örnek lüks özellik dağılımı:\n", df_final['has_luxury'].value_counts())
df_final.head()

Toplam 26 özellik parse edildi.
Örnek lüks özellik dağılımı:
 has_luxury
False    6223
True      614
Name: count, dtype: int64


,id,name,price,rooms,bath,area_m2,location,lat,long,building_no,amenities,furnished,reactivatedat,city,region,district,link,type,beds,postal_code,additional_no,isverified,listing_age,listing_face,deed_number,is_listing_constrained,is_listing_pawned,agency_name,residence_type,h3_index,h3_11,property_id,total_listings_count,is_faulty,is_main_listing,has_fiber_optics,has_security,has_parking_spaces,has_flood_drainage,has_cctv_security,has_terrace,has_mosque,has_sewerage,has_close_to_main_roads,has_hospitals,has_driver,has_electricity,has_water_supply,has_shopping_malls,has_smart_home,has_pool,has_gym,has_maid,has_atm_facility,has_elevator,has_schools,has_balcony,has_view,has_fixed_phone,has_garden,has_elevators_in_building,has_luxury
1492,87862939,Apartment for sale in Al-Mahdiyah neighborhood,880000.0,4,3,127.42,Al Mahdiyah,24.654275,46.515877,2322,"Electricity, Water Supply, Sewerage, Fiber Opt...",unfurnished,2026-01-07,Riyadh,West Riyadh,Al Mahdiyah,https://www.bayut.sa/en/property/details-87862...,developer,4,13754,7422,False,New,Southern,5878890131400010,No,No,MUAFQA REAL ESTATE COMPANY,NaN,895354a542bffff,8b5354a5429efff,4311,2,False,True,True,False,False,False,False,False,True,True,False,False,False,True,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False
950,87862909,Spacious apartments with a terrace for each ap...,1200000.0,6,4,230.56,Al Salamah,21.609659,39.147102,7783,"Private Entrance, Electricity, Water Supply, S...",unfurnished,2026-01-07,Jeddah,North Jeddah,Al Salamah,https://www.bayut.sa/en/property/details-87862...,agency,6,23525,3015,False,New,NaN,860002610239,No,No,Almurad real estate,NaN,8953a952d4fffff,8b53a952d4c5fff,6106,1,False,True,True,True,False,True,True,True,True,True,True,True,False,True,True,True,False,False,False,True,False,True,True,True,False,True,False,True,False
1693,87862817,"4 Bedroom Apartment For Sale in Al Salam, Jeddah",650000.0,4,3,111.49,Al Salamah,21.592591,39.157788,7984,"Electricity, Water Supply, Sewerage",unfurnished,2026-01-07,Jeddah,North Jeddah,Al Salamah,https://www.bayut.sa/en/property/details-87862...,agency,4,23437,3494,False,New,NaN,160002909846,No,No,Fikr Al E'mar Company,family,8953a95290bffff,8b53a9529096fff,266,28,False,True,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1694,87862815,"4 Bedroom Apartment For Sale in Al Rawda, Jeddah",780000.0,4,2,132.26,Al Rawdah,21.565379,39.160321,7873,"Electricity, Water Supply, Sewerage",unfurnished,2026-01-07,Jeddah,North Jeddah,Al Rawdah,https://www.bayut.sa/en/property/details-87862...,agency,3,23433,3832,False,New,NaN,960001396609,No,No,Fikr Al E'mar Company,family,8953a950db3ffff,8b53a950db26fff,6737,6,False,True,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1695,87862735,4-bedroom apartment for sale in Badr. For comm...,670000.0,4,5,200.13,Badr,24.513676,46.729632,2880,"Electricity, Water Supply, Sewerage",unfurnished,2026-01-07,Riyadh,South Riyadh,Badr,https://www.bayut.sa/en/property/details-87862...,agency,4,14743,6574,False,New,Eastern,260026300383,No,No,Wathaek Real Estate Development Company,family,895355c302bffff,8b5355c302adfff,836,9,False,True,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [17]:
# --- HIBRIT OUTLIER FILTLEME (FİNAL: SADELEŞTİRİLMİŞ & GÜVENLİ) ---
import numpy as np

def compute_group_quantiles(df, filters, iqr_multiplier=2.0):
    quantiles = {}
    for filt in filters:
        target_col = filt['column']
        group_cols = filt['group_cols']
        
        # Gruplama için eksik verileri 'Unknown' veya -1 ile doldur
        temp_df = df.copy()
        for col in group_cols:
            if temp_df[col].dtype == 'object':
                temp_df[col] = temp_df[col].fillna('Unknown')
            else:
                temp_df[col] = temp_df[col].fillna(-1)

        group_stats = temp_df.groupby(group_cols)[target_col].agg(
            Q1=lambda x: x.quantile(0.25),
            Q3=lambda x: x.quantile(0.75),
            count='count'
        ).reset_index()

        # Sadece yeterli verisi olan gruplarda IQR uygula (en az 3 veri)
        group_stats['IQR'] = group_stats['Q3'] - group_stats['Q1']
        group_stats.loc[group_stats['count'] < 3, 'IQR'] = np.nan
        
        # GEVŞETİLMİŞ FİLTRE: 1.5 yerine, lüks evleri korumak için daha geniş bir aralık
        group_stats['lower_bound'] = group_stats['Q1'] - iqr_multiplier * group_stats['IQR']
        group_stats['upper_bound'] = group_stats['Q3'] + iqr_multiplier * group_stats['IQR']

        quantiles[target_col] = group_stats
    return quantiles

def apply_group_iqr_filter(df, quantiles, filters):
    df_filtered = df.copy()
    for filt in filters:
        target_col = filt['column']
        group_cols = filt['group_cols']
        
        # Merge için geçici doldurma
        temp_df = df_filtered.copy()
        for col in group_cols:
            if temp_df[col].dtype == 'object':
                temp_df[col] = temp_df[col].fillna('Unknown')
            else:
                temp_df[col] = temp_df[col].fillna(-1)

        thresholds = quantiles[target_col]
        df_merged = pd.merge(
            temp_df,
            thresholds[group_cols + ['lower_bound', 'upper_bound']],
            on=group_cols,
            how='left'
        )

        mask = (
            (df_merged[target_col] >= df_merged['lower_bound']) | df_merged['lower_bound'].isnull()
        ) & (
            (df_merged[target_col] <= df_merged['upper_bound']) | df_merged['upper_bound'].isnull()
        )

        df_filtered = df_filtered[mask.values].reset_index(drop=True)
        atilan = len(df) - len(df_filtered)
        print(f"'{target_col}' filtresiyle atılan outlier sayısı: {atilan}")
        
    return df_filtered

# SADELEŞTİRİLMİŞ FİLTRE KRİTERLERİ (Amenities gruplamadan çıkarıldı)
outlier_filters = [
    {'column': 'price', 'group_cols': ['rooms', 'h3_index']},
    {'column': 'area_m2', 'group_cols': ['rooms', 'h3_index']}
]

# GÜVENLİK AYARI: IQR çarpanı 2.0 yapıldı (Lüks/Standart karışık gruplar için koruma)
print(f"IQR Öncesi: {len(df_final)}")
q_thresholds = compute_group_quantiles(df_final, outlier_filters, iqr_multiplier=2.0)
df_final = apply_group_iqr_filter(df_final, q_thresholds, outlier_filters)
print(f"FİNAL Temiz Veri (Gevşetilmiş Filtre 2.0x): {len(df_final)}")

IQR Öncesi: 6837
'price' filtresiyle atılan outlier sayısı: 104
'area_m2' filtresiyle atılan outlier sayısı: 186
FİNAL Temiz Veri (Gevşetilmiş Filtre 2.0x): 6651


In [18]:
# --- NİHAİ SÜTUN SIRALAMASI ---
desired_cols = [
    'price', 'rooms', 'beds', 'area_m2', 'deed_number', 'building_no', 
    'additional_no', 'postal_code', 'lat', 'long', 'property_id', 'total_listings_count', 'is_faulty', 'is_main_listing', 'link', 
    'id', 'h3_11', 'name', 'bath', 'location', 'city', 'region', 'district', 
    'reactivatedat', 'listing_age', 'listing_face', 'type', 'isverified', 
    'furnished', 'amenities', 'agency_name', 'residence_type', 'is_listing_constrained', 'is_listing_pawned', 'h3_index'
]

# Mevcut olmayan sütunları (varsa) listeden çıkaralım
existing_cols = [c for c in desired_cols if c in df_final.columns]
# Listede olmayan ama df'de olan sütunları sona ekleyelim
other_cols = [c for c in df_final.columns if c not in existing_cols]

df_final = df_final[existing_cols + other_cols]

print("Sütunlar başarıyla yeniden sıralandı.")
df_final.head()

Sütunlar başarıyla yeniden sıralandı.


,price,rooms,beds,area_m2,deed_number,building_no,additional_no,postal_code,lat,long,property_id,total_listings_count,is_faulty,is_main_listing,link,id,h3_11,name,bath,location,city,region,district,reactivatedat,listing_age,listing_face,type,isverified,furnished,amenities,agency_name,residence_type,is_listing_constrained,is_listing_pawned,h3_index,has_fiber_optics,has_security,has_parking_spaces,has_flood_drainage,has_cctv_security,has_terrace,has_mosque,has_sewerage,has_close_to_main_roads,has_hospitals,has_driver,has_electricity,has_water_supply,has_shopping_malls,has_smart_home,has_pool,has_gym,has_maid,has_atm_facility,has_elevator,has_schools,has_balcony,has_view,has_fixed_phone,has_garden,has_elevators_in_building,has_luxury
0,880000.0,4,4,127.42,5878890131400010,2322,7422,13754,24.654275,46.515877,4311,2,False,True,https://www.bayut.sa/en/property/details-87862...,87862939,8b5354a5429efff,Apartment for sale in Al-Mahdiyah neighborhood,3,Al Mahdiyah,Riyadh,West Riyadh,Al Mahdiyah,2026-01-07,New,Southern,developer,False,unfurnished,"Electricity, Water Supply, Sewerage, Fiber Opt...",MUAFQA REAL ESTATE COMPANY,NaN,No,No,895354a542bffff,True,False,False,False,False,False,True,True,False,False,False,True,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False
1,1200000.0,6,6,230.56,860002610239,7783,3015,23525,21.609659,39.147102,6106,1,False,True,https://www.bayut.sa/en/property/details-87862...,87862909,8b53a952d4c5fff,Spacious apartments with a terrace for each ap...,4,Al Salamah,Jeddah,North Jeddah,Al Salamah,2026-01-07,New,NaN,agency,False,unfurnished,"Private Entrance, Electricity, Water Supply, S...",Almurad real estate,NaN,No,No,8953a952d4fffff,True,True,False,True,True,True,True,True,True,True,False,True,True,True,False,False,False,True,False,True,True,True,False,True,False,True,False
2,650000.0,4,4,111.49,160002909846,7984,3494,23437,21.592591,39.157788,266,28,False,True,https://www.bayut.sa/en/property/details-87862...,87862817,8b53a9529096fff,"4 Bedroom Apartment For Sale in Al Salam, Jeddah",3,Al Salamah,Jeddah,North Jeddah,Al Salamah,2026-01-07,New,NaN,agency,False,unfurnished,"Electricity, Water Supply, Sewerage",Fikr Al E'mar Company,family,No,No,8953a95290bffff,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,780000.0,4,3,132.26,960001396609,7873,3832,23433,21.565379,39.160321,6737,6,False,True,https://www.bayut.sa/en/property/details-87862...,87862815,8b53a950db26fff,"4 Bedroom Apartment For Sale in Al Rawda, Jeddah",2,Al Rawdah,Jeddah,North Jeddah,Al Rawdah,2026-01-07,New,NaN,agency,False,unfurnished,"Electricity, Water Supply, Sewerage",Fikr Al E'mar Company,family,No,No,8953a950db3ffff,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,670000.0,4,4,200.13,260026300383,2880,6574,14743,24.513676,46.729632,836,9,False,True,https://www.bayut.sa/en/property/details-87862...,87862735,8b5355c302adfff,4-bedroom apartment for sale in Badr. For comm...,5,Badr,Riyadh,South Riyadh,Badr,2026-01-07,New,Eastern,agency,False,unfurnished,"Electricity, Water Supply, Sewerage",Wathaek Real Estate Development Company,family,No,No,895355c302bffff,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [19]:
# --- SONUÇ: ML İÇİN HAZIR VERİ ---
# 1. binary sütunları (True/False) 1/0 a çevirelim ki model anlasın
bool_cols = [col for col in df_final.columns if col.startswith('has_') or col == 'isverified']
for col in bool_cols:
    df_final[col] = df_final[col].astype(int)

print("Veri seti ML modellemesi için hazırlandı.")
df_final.to_csv("../data/processed/df_final_ml.csv", index=False, encoding="utf-8-sig")
print("final_data_for_ml.csv olarak kaydedildi.")


Veri seti ML modellemesi için hazırlandı.
final_data_for_ml.csv olarak kaydedildi.
